In [1]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

2024-02-20 15:14:22.742881: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-20 15:14:22.785183: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-20 15:14:23.495953: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import numpy as np
import cv2  # OpenCVライブラリ
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import os
import sys
import pathlib
from pathlib import Path

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
import swallowing_recognition

In [4]:
current_file_path = pathlib.Path(os.getcwd())
parent_dir = current_file_path.parent

In [5]:
from swallowing_recognition import wavelet
from swallowing_recognition import audio
from swallowing_recognition import dataset
from swallowing_recognition import dence_net

In [6]:
directory_path = parent_dir / 'dataset'
   
train_voice_folder = directory_path / 'washino' / 'voice'
train_cough_folder = directory_path / 'washino' / 'cough'
train_swallowing_folder = directory_path / 'washino' / 'swallowing'   
train_neck_movement_folder = directory_path / 'washino' / 'neck_movement'
train_shoulder_movement_folder = directory_path / 'washino' / 'shoulder_movement'
train_arm_movement_folder = directory_path / 'washino' / 'arm_movement'   
train_hitting_folder = directory_path / 'washino' / 'hitting'   
train_mumble_folder = directory_path / 'washino' / 'mumble'   

test_voice_folder = directory_path / 'shibata' / 'voice'
test_cough_folder = directory_path / 'shibata' / 'cough'
test_swallowing_folder = directory_path / 'shibata' / 'swallowing'    
test_movement_folder = directory_path / 'shibata' / 'movement'    
test_hitting_folder = directory_path / 'shibata' / 'hitting'    

# 引数は(データ数，画像サイズ縦，画像サイズ横，色RGB，クラス数)
# train_data = dataset.DataSet(500, 224, 224, 3, 5)
# test_data = dataset.DataSet(70, 224, 224, 3, 5)

train_data = dataset.DataSet(1400, 224, 224, 3, 2)
test_data = dataset.DataSet(62, 224, 224, 3, 2)

# train_data.folder_to_dataset(train_voice_folder, np.array([0, 0, 0, 0, 1]), 0)
# train_data.folder_to_dataset(train_cough_folder, np.array([0, 0, 0, 1, 0]), 1)
# train_data.folder_to_dataset(train_swallowing_folder, np.array([0, 0, 1, 0, 0]), 2)
# train_data.folder_to_dataset(train_movement_folder, np.array([0, 1, 0, 0, 0]), 3)
# train_data.folder_to_dataset(train_hitting_folder, np.array([1, 0, 0, 0, 0]), 4)

train_data.folder_to_dataset(train_swallowing_folder, np.array(0), 0)

In [7]:
train_data.folder_to_dataset(train_voice_folder, np.array(1), 700)
train_data.folder_to_dataset(train_neck_movement_folder, np.array(1), 800)
train_data.folder_to_dataset(train_shoulder_movement_folder, np.array(1), 900)
train_data.folder_to_dataset(train_arm_movement_folder, np.array(1), 1000)
train_data.folder_to_dataset(train_hitting_folder, np.array(1), 1100)
train_data.folder_to_dataset(train_cough_folder, np.array(1), 1200)
train_data.folder_to_dataset(train_mumble_folder, np.array(1), 1300)

In [8]:
test_data.folder_to_dataset(test_swallowing_folder, np.array(0), 0)
test_data.folder_to_dataset(test_cough_folder, np.array(1), 14)
test_data.folder_to_dataset(test_voice_folder, np.array(1), 28)
test_data.folder_to_dataset(test_movement_folder, np.array(1), 42)
test_data.folder_to_dataset(test_hitting_folder, np.array(1), 52)

In [9]:
print(type(train_data.data))
print((train_data.data).shape)
print((train_data.labels).shape)

<class 'numpy.ndarray'>
(1400, 224, 224, 3)
(1400,)


In [28]:
# model = dence_net.DanceNet(5)

model = dence_net.DanceNet(2)

early_stopping = EarlyStopping(
    monitor='val_loss',  # 監視する値
    patience=10,         # 性能が改善されないエポック数
    verbose=1,           # 進行状況のメッセージ表示
    mode='min',          # 「減少」を監視
    restore_best_weights=True  # 最良のモデルの重みを復元
)

model_checkpoint = ModelCheckpoint(
    filepath='20240221_binary_model_best.keras',  # 保存するファイルのパス
    monitor='val_loss',        # 監視する値
    save_best_only=True,       # 最良のモデルのみを保存
    verbose=1,                 # 進行状況のメッセージ表示
    mode='min'                 # 「減少」を監視
)

# model_checkpoint = ModelCheckpoint(
#     filepath='20240213_binary_model_best.keras',  # 保存するファイルのパス
#     monitor = "accuracy",
#     save_best_only=True,       # 最良のモデルのみを保存
#     verbose=1,                 # 進行状況のメッセージ表示
#     mode='max'                 # 「減少」を監視
# )

# 引数は(トレーニングデータ，ラベル，epochs，batch_size)
model.training(train_data.data, train_data.labels, 50, 32, model_checkpoint = model_checkpoint)
model.evaluate(test_data.data, test_data.labels)
model.save('20240221_binary_ep50_bs32_v2.keras')

Epoch 1/50
40/40 [==============================] - ETA: 0s - loss: 0.2973 - accuracy: 0.8984
Epoch 1: val_loss improved from inf to 28.95901, saving model to 20240220_binary_model_best.keras
40/40 [==============================] - 55s 160ms/step - loss: 0.2973 - accuracy: 0.8984 - val_loss: 28.9590 - val_accuracy: 0.0429
Epoch 2/50
40/40 [==============================] - ETA: 0s - loss: 0.1488 - accuracy: 0.9437
Epoch 2: val_loss improved from 28.95901 to 12.69418, saving model to 20240220_binary_model_best.keras
40/40 [==============================] - 5s 128ms/step - loss: 0.1488 - accuracy: 0.9437 - val_loss: 12.6942 - val_accuracy: 0.4929
Epoch 3/50
40/40 [==============================] - ETA: 0s - loss: 0.1235 - accuracy: 0.9603
Epoch 3: val_loss improved from 12.69418 to 0.61444, saving model to 20240220_binary_model_best.keras
40/40 [==============================] - 5s 125ms/step - loss: 0.1235 - accuracy: 0.9603 - val_loss: 0.6144 - val_accuracy: 0.8929
Epoch 4/50
40/40 [=

Epoch 30/50
40/40 [==============================] - ETA: 0s - loss: 0.0187 - accuracy: 0.9921
Epoch 30: val_loss did not improve from 0.09400
40/40 [==============================] - 4s 105ms/step - loss: 0.0187 - accuracy: 0.9921 - val_loss: 0.4479 - val_accuracy: 0.8929
Epoch 31/50
40/40 [==============================] - ETA: 0s - loss: 0.0083 - accuracy: 0.9984
Epoch 31: val_loss did not improve from 0.09400
40/40 [==============================] - 4s 106ms/step - loss: 0.0083 - accuracy: 0.9984 - val_loss: 0.2760 - val_accuracy: 0.9286
Epoch 32/50
40/40 [==============================] - ETA: 0s - loss: 0.0137 - accuracy: 0.9944
Epoch 32: val_loss improved from 0.09400 to 0.07535, saving model to 20240220_binary_model_best.keras
40/40 [==============================] - 5s 122ms/step - loss: 0.0137 - accuracy: 0.9944 - val_loss: 0.0754 - val_accuracy: 0.9786
Epoch 33/50
40/40 [==============================] - ETA: 0s - loss: 0.0055 - accuracy: 0.9984
Epoch 33: val_loss did not im

In [11]:
# model.evaluate_print()
test_data2 = dataset.DataSet(42, 224, 224, 3, 2)
test_data2.folder_to_dataset(test_swallowing_folder, np.array(0), 0)
test_data2.folder_to_dataset(test_cough_folder, np.array(1), 14)
test_data2.folder_to_dataset(test_voice_folder, np.array(1), 28)

In [12]:
model.evaluate(test_data2.data, test_data2.labels)

2/2 [==============================] - 0s 138ms/step - loss: 0.2215 - accuracy: 0.9048
Test accuracy:  0.9047619104385376
2/2 [==============================] - 0s 24ms/step
正しく分類されたサンプルのインデックス: [ 0  1  2  3  5  6  7  8  9 10 11 12 14 15 16 18 19 20 21 22 23 24 25 26
 27 28 29 30 31 32 33 34 35 37 38 39 40 41]
誤って分類されたサンプルのインデックス: [ 4 13 17 36]
サンプル 4: 正解 = 0.0, 予測 = 1
サンプル 13: 正解 = 0.0, 予測 = 1
サンプル 17: 正解 = 1.0, 予測 = 0
サンプル 36: 正解 = 1.0, 予測 = 0
